In [741]:
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime

people = ['tyler', 'hotdog', 'chris', 'ryan']
selection = 3
    

In [742]:
df = pd.read_csv('output/cage.csv', index_col=0)
df = df.drop(columns=['Screen Time Lower', 'Sample', 'Total', 'Appears'])
# df.head()

In [743]:
time_cols = ['Duration', 'Face Time', 'Screen Time Upper']

for t in time_cols:
    df[t] = df[t].apply(
        lambda x: (datetime.strptime(x, '%H:%M:%S') - datetime(1900, 1, 1)).total_seconds()
    )

df['Year'] = df['Year'].apply(lambda x: int(x - 1982))

# df.head()

In [744]:
# columsn used to make prediction
X = df.loc[:, ['Duration', 'IMDB', 'Rotten', 'Year']]
# column trying to predict
y = df.iloc[:, selection]

# split between training and testing data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, # random_state = 69
)

In [745]:
"""
sns.pairplot(
    pd.merge(X, y, left_index=True, right_index=True, how='outer'), 
    #hue=people[selection].title(),
    palette='rocket_r',
    corner=True,
    diag_kind='hist',
)
"""

"\nsns.pairplot(\n    pd.merge(X, y, left_index=True, right_index=True, how='outer'), \n    #hue=people[selection].title(),\n    palette='rocket_r',\n    corner=True,\n    diag_kind='hist',\n)\n"

In [746]:
# clf = RandomForestRegressor()
# clf = ExtraTreesRegressor()
clf = AdaBoostRegressor()

clf.fit(X_train, y_train)

AdaBoostRegressor()

In [747]:
train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)

train_mse = mean_squared_error(y_train, train_pred)
train_r2 = r2_score(y_train, train_pred)

test_mse = mean_squared_error(y_test, test_pred)
test_r2 = r2_score(y_test, test_pred)

In [748]:
results = pd.DataFrame(['clf', train_mse, train_r2, test_mse, test_r2]).transpose()
results.columns = ['Method', 'Training MSE', 'Training R2', 'Test MSE', 'Test R2']
results.head()

,Method,Training MSE,Training R2,Test MSE,Test R2
0,clf,0.010254,0.995061,2.70875,0.452691


In [749]:
with open(f'output/{people[selection]}.pickle', 'wb') as f:
    pickle.dump(clf, f)